In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.interpolate import interp1d
import plotly.graph_objects as go

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [4]:
US_testing_series_df = pd.read_csv('../optional_datasets/COVID-19-JohnHopkins/data_tables/testing_data/time_series_covid19_US.csv')
US_testing_series_df

,date,state,cases_conf_probable,cases_confirmed,cases_probable,tests_viral_positive,tests_viral_negative,tests_viral_total,tests_antigen_positive,tests_antigen_total,people_viral_positive,people_viral_total,people_antigen_positive,people_antigen_total,encounters_viral_total,tests_combined_total
0,3/6/2020,AK,NaN,NaN,NaN,0.0,27.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0
1,3/7/2020,AK,NaN,NaN,NaN,0.0,32.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0
2,3/8/2020,AK,NaN,NaN,NaN,0.0,40.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0
3,3/9/2020,AK,NaN,NaN,NaN,0.0,59.0,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0
4,3/10/2020,AK,NaN,NaN,NaN,0.0,76.0,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49167,7/27/2022,VT,137649.0,129271.0,8378.0,NaN,NaN,3764554.0,NaN,NaN,129271.0,NaN,NaN,NaN,NaN,3764554.0
49168,7/27/2022,WA,1729949.0,1729949.0,0.0,NaN,NaN,NaN,NaN,NaN,1729949.0,NaN,NaN,NaN,NaN,NaN
49169,7/27/2022,WI,1773092.0,1561247.0,211845.0,1673574.0,15594942.0,17268516.0,NaN,NaN,1561247.0,NaN,NaN,NaN,17268516.0,17268516.0
49170,7/27/2022,WV,553018.0,431759.0,121259.0,NaN,NaN,6537070.0,NaN,NaN,431759.0,NaN,NaN,NaN,NaN,6658329.0


In [5]:
US_testing_series_df.isna().sum()

date                           0
state                          0
cases_conf_probable          336
cases_confirmed            22523
cases_probable             22573
tests_viral_positive       26672
tests_viral_negative       35548
tests_viral_total          13776
tests_antigen_positive     45736
tests_antigen_total        35433
people_viral_positive       4983
people_viral_total         31227
people_antigen_positive    47155
people_antigen_total       46451
encounters_viral_total     37971
tests_combined_total        3242
dtype: int64

In [15]:
US_testing_series_df.describe()

,cases_conf_probable,cases_confirmed,cases_probable,tests_viral_positive,tests_viral_negative,tests_viral_total,tests_antigen_positive,tests_antigen_total,people_viral_positive,people_viral_total,people_antigen_positive,people_antigen_total,encounters_viral_total,tests_combined_total
count,4.883600e+04,2.664900e+04,2.659900e+04,2.250000e+04,1.362400e+04,3.539600e+04,3.436000e+03,1.373900e+04,4.418900e+04,1.794500e+04,2017.000000,2.721000e+03,1.120100e+04,4.593000e+04
mean,6.432550e+05,9.192764e+05,1.058146e+05,6.750425e+05,5.357850e+06,8.436064e+06,2.104479e+05,9.615107e+05,6.060728e+05,2.032355e+06,109639.408528,6.143451e+05,8.980778e+06,8.326903e+06
std,1.105653e+06,1.257586e+06,1.739846e+05,1.100801e+06,7.836575e+06,1.663510e+07,2.686045e+05,1.897234e+06,1.056232e+06,2.727065e+06,88308.994134,5.147278e+05,1.741531e+07,1.700022e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,3.000000,3.000000e+00,0.000000e+00,0.000000e+00
25%,4.311425e+04,1.893070e+05,0.000000e+00,7.220200e+04,1.147190e+06,9.329618e+05,3.274000e+04,6.793300e+04,5.322700e+04,3.235810e+05,51138.000000,2.223080e+05,1.054962e+06,7.204505e+05
50%,2.407995e+05,5.195440e+05,2.609600e+04,2.854985e+05,3.187079e+06,3.227059e+06,9.722250e+04,2.919650e+05,2.378120e+05,1.020216e+06,81803.000000,4.715680e+05,3.579350e+06,2.897050e+06
75%,7.764358e+05,1.112491e+06,1.483250e+05,8.038925e+05,6.381068e+06,9.148805e+06,3.096810e+05,9.548450e+05,7.037900e+05,2.792464e+06,162042.000000,9.529010e+05,9.525773e+06,9.048939e+06
max,1.063206e+07,9.871930e+06,1.370689e+06,7.599635e+06,5.334933e+07,1.787442e+08,1.334604e+06,1.558732e+07,9.871930e+06,1.833717e+07,296030.000000,1.880643e+06,1.155660e+08,1.787442e+08


In [3]:
US_county_series_df = pd.read_csv('../optional_datasets/COVID-19-JohnHopkins/data_tables/testing_data/county_time_series_covid19_US.csv')
US_county_series_df

,fips,lat,long,county,province_state,combined_key,date,tests_combined_total
0,1001.0,32.5395,-86.6441,Autauga,Alabama,"Autauga, Alabama, US",2021-08-01,29714.0
1,1003.0,30.7277,-87.7221,Baldwin,Alabama,"Baldwin, Alabama, US",2021-08-01,118240.0
2,1005.0,31.8683,-85.3871,Barbour,Alabama,"Barbour, Alabama, US",2021-08-01,12471.0
3,1007.0,32.9964,-87.1251,Bibb,Alabama,"Bibb, Alabama, US",2021-08-01,14814.0
4,1009.0,33.9821,-86.5679,Blount,Alabama,"Blount, Alabama, US",2021-08-01,27943.0
...,...,...,...,...,...,...,...,...
571376,56041.0,41.2878,-110.5476,Uinta,Wyoming,"Uinta, Wyoming, US",2022-07-23,37048.0
571377,56043.0,43.9045,-107.6802,Washakie,Wyoming,"Washakie, Wyoming, US",2022-07-23,18016.0
571378,56045.0,43.8396,-104.5675,Weston,Wyoming,"Weston, Wyoming, US",2022-07-23,14460.0
571379,80056.0,NaN,NaN,Out of WY,Wyoming,"Out of WY, Wyoming, US",2022-07-23,NaN


In [6]:
US_county_series_df.isna().sum()

fips                      584
lat                     15978
long                    15978
county                      0
province_state              0
combined_key                0
date                        0
tests_combined_total    23240
dtype: int64

In [16]:
US_county_series_df.describe()

,fips,lat,long,tests_combined_total
count,570797.000000,555403.000000,555403.000000,5.481410e+05
mean,34359.455757,38.637565,-92.127396,2.631405e+05
std,18430.157887,5.470005,13.442713,1.368958e+06
min,1001.000000,19.601200,-174.159600,-4.810457e+06
25%,19091.000000,34.811900,-96.487600,1.552000e+04
50%,36015.000000,38.716200,-90.710100,4.282700e+04
75%,48247.000000,42.383800,-84.520800,1.310610e+05
max,90056.000000,69.314800,-71.023400,7.077137e+07


In [8]:
vaccine_us_timeline_df= pd.read_csv('../optional_datasets/COVID-19-JohnHopkins/data_tables/vaccine_data/us_data/time_series/vaccine_data_us_timeline.csv')
vaccine_us_timeline_df

,Province_State,Date,Vaccine_Type,FIPS,Country_Region,Lat,Long_,Doses_alloc,Doses_shipped,Doses_admin,Stage_One_Doses,Stage_Two_Doses,Combined_Key
0,Alabama,2020-12-10,All,1.0,US,32.3182,-86.9023,NaN,NaN,NaN,NaN,NaN,"Alabama, US"
1,Alabama,2020-12-10,Moderna,1.0,US,32.3182,-86.9023,NaN,NaN,NaN,NaN,NaN,"Alabama, US"
2,Alabama,2020-12-10,Pfizer,1.0,US,32.3182,-86.9023,NaN,NaN,NaN,NaN,NaN,"Alabama, US"
3,Alabama,2020-12-11,All,1.0,US,32.3182,-86.9023,NaN,NaN,NaN,NaN,NaN,"Alabama, US"
4,Alabama,2020-12-11,Moderna,1.0,US,32.3182,-86.9023,NaN,NaN,NaN,NaN,NaN,"Alabama, US"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
205181,Wyoming,2022-07-27,Moderna,56.0,US,42.7560,-107.3025,NaN,465740.0,339592.0,NaN,121052.0,"Wyoming, US"
205182,Wyoming,2022-07-27,Unknown,56.0,US,42.7560,-107.3025,NaN,0.0,733.0,NaN,NaN,"Wyoming, US"
205183,Wyoming,2022-07-27,Janssen,56.0,US,42.7560,-107.3025,NaN,48700.0,28924.0,26889.0,NaN,"Wyoming, US"
205184,Wyoming,2022-07-27,All,56.0,US,42.7560,-107.3025,NaN,1061485.0,768328.0,340904.0,278129.0,"Wyoming, US"


In [10]:
vaccine_us_timeline_df.isna().sum()

Province_State          0
Date                    0
Vaccine_Type            0
FIPS                17442
Country_Region          0
Lat                 17442
Long_               17442
Doses_alloc        185425
Doses_shipped       31137
Doses_admin         27267
Stage_One_Doses    103063
Stage_Two_Doses     86918
Combined_Key            0
dtype: int64

In [17]:
vaccine_us_timeline_df.describe()

,FIPS,Lat,Long_,Doses_alloc,Doses_shipped,Doses_admin,Stage_One_Doses,Stage_Two_Doses
count,187744.000000,187744.000000,187744.000000,1.976100e+04,1.740490e+05,1.779190e+05,1.021230e+05,1.182680e+05
mean,32.538712,36.846779,-85.178368,2.793917e+06,3.181236e+06,3.609387e+06,2.222517e+06,1.454839e+06
std,18.909834,10.792400,49.393707,6.021776e+06,7.247861e+06,2.549376e+07,4.039957e+06,2.612688e+06
min,1.000000,-14.271000,-170.132000,0.000000e+00,-1.860390e+05,-2.713460e+05,-1.538000e+03,-2.126800e+05
25%,17.000000,33.856900,-105.311100,4.680000e+04,0.000000e+00,3.487000e+03,1.480680e+05,6.851575e+04
50%,32.000000,39.063900,-86.902300,3.030650e+05,4.740900e+05,3.218780e+05,6.669710e+05,5.424475e+05
75%,47.000000,42.756000,-77.026800,1.812025e+06,3.264030e+06,2.625104e+06,2.665860e+06,1.755892e+06
max,78.000000,61.370700,145.673900,2.751374e+07,9.718966e+07,5.879034e+08,3.410072e+07,2.773966e+07


In [9]:
vacc_idv_timeline_df= pd.read_csv('../optional_datasets/COVID-19-JohnHopkins/data_tables/vaccine_data/us_data/time_series/people_vaccinated_us_timeline.csv')
vacc_idv_timeline_df

,FIPS,Province_State,Country_Region,Date,Lat,Long_,Combined_Key,People_Fully_Vaccinated,People_Partially_Vaccinated
0,1.0,Alabama,US,2020-12-10,32.3182,-86.9023,"Alabama, US",NaN,NaN
1,1.0,Alabama,US,2020-12-11,32.3182,-86.9023,"Alabama, US",NaN,NaN
2,1.0,Alabama,US,2020-12-12,32.3182,-86.9023,"Alabama, US",NaN,NaN
3,1.0,Alabama,US,2020-12-13,32.3182,-86.9023,"Alabama, US",NaN,NaN
4,1.0,Alabama,US,2020-12-14,32.3182,-86.9023,"Alabama, US",NaN,NaN
...,...,...,...,...,...,...,...,...,...
36007,56.0,Wyoming,US,2022-07-23,42.7560,-107.3025,"Wyoming, US",298235.0,42669.0
36008,56.0,Wyoming,US,2022-07-24,42.7560,-107.3025,"Wyoming, US",298235.0,42669.0
36009,56.0,Wyoming,US,2022-07-25,42.7560,-107.3025,"Wyoming, US",298235.0,42669.0
36010,56.0,Wyoming,US,2022-07-26,42.7560,-107.3025,"Wyoming, US",298235.0,42669.0


In [11]:
vacc_idv_timeline_df.isna().sum()

FIPS                           2692
Province_State                    0
Country_Region                    0
Date                              0
Lat                            2692
Long_                          2692
Combined_Key                      0
People_Fully_Vaccinated        2359
People_Partially_Vaccinated    3335
dtype: int64

In [13]:
vacc_admn_timeseries_df = pd.read_csv('../optional_datasets/COVID-19-JohnHopkins/data_tables/vaccine_data/us_data/time_series/time_series_covid19_vaccine_doses_admin_US.csv')
vacc_admn_timeseries_df

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2022-07-18,2022-07-19,2022-07-20,2022-07-21,2022-07-22,2022-07-23,2022-07-24,2022-07-25,2022-07-26,2022-07-27
0,84000001.0,US,USA,840.0,1.0,NaN,Alabama,US,32.3182,-86.9023,...,6438626.0,6438626.0,6438626.0,6463818.0,6463818.0,6463818.0,6463818.0,6463818.0,6463818.0,6463818.0
1,84000002.0,US,USA,840.0,2.0,NaN,Alaska,US,61.3707,-152.4044,...,1181814.0,1181814.0,1181814.0,1186419.0,1186419.0,1186419.0,1186419.0,1186419.0,1186419.0,1186419.0
2,84000004.0,US,USA,840.0,4.0,NaN,Arizona,US,33.7298,-111.4312,...,12668515.0,12668515.0,12668515.0,12727134.0,12727134.0,12727134.0,12727134.0,12727134.0,12727134.0,12727134.0
3,84000005.0,US,USA,840.0,5.0,NaN,Arkansas,US,34.9697,-92.3731,...,4372933.0,4372933.0,4372933.0,4373349.0,4373349.0,4373349.0,4373349.0,4373349.0,4373349.0,4373349.0
4,84000006.0,US,USA,840.0,6.0,NaN,California,US,36.1162,-119.6816,...,78269641.0,78269641.0,78319081.0,78558967.0,78558967.0,78558967.0,78558967.0,78567829.0,78583471.0,78621316.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,NaN,NaN,NaN,NaN,NaN,NaN,Department of Defense,US,NaN,NaN,...,7560317.0,7560317.0,8377268.0,8417991.0,8417991.0,8417991.0,8417991.0,8417991.0,8417991.0,8417991.0
57,NaN,NaN,NaN,NaN,NaN,NaN,Federal Bureau of Prisons,US,NaN,NaN,...,325739.0,325739.0,325739.0,325752.0,325752.0,325752.0,325752.0,325752.0,325752.0,325752.0
58,NaN,NaN,NaN,NaN,NaN,NaN,Indian Health Services,US,NaN,NaN,...,2310967.0,2310967.0,2310967.0,2318179.0,2318179.0,2318179.0,2318179.0,2318179.0,2318179.0,2318179.0
59,NaN,NaN,NaN,NaN,NaN,NaN,Long Term Care (LTC) Program,US,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
vacc_admn_timeseries_df.isna().sum()

UID           5
iso2          5
iso3          5
code3         5
FIPS          5
             ..
2022-07-23    1
2022-07-24    1
2022-07-25    1
2022-07-26    1
2022-07-27    1
Length: 603, dtype: int64

In [19]:
cases_by_state_df = pd.read_csv('../optional_datasets/COVID-19-JohnHopkins/data_tables/JHU_USCountymap_TEST/df_Counties2020.csv')
cases_by_state_df

,Unnamed: 0,Countyname,ST_Name,FIPS,ST_ID,dt,Confirmed,Deaths,Population,IncidenceRate,NewCases
0,0,Autauga,Alabama,1001,1,2020-01-22,0,0,55869,0.0,0.0
1,3342,Autauga,Alabama,1001,1,2020-01-23,0,0,55869,0.0,0.0
2,6684,Autauga,Alabama,1001,1,2020-01-24,0,0,55869,0.0,0.0
3,10026,Autauga,Alabama,1001,1,2020-01-25,0,0,55869,0.0,0.0
4,13368,Autauga,Alabama,1001,1,2020-01-26,0,0,55869,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1547341,1533975,Unassigned,Wyoming,90056,0,2021-04-24,0,0,0,0.0,0.0
1547342,1537317,Unassigned,Wyoming,90056,0,2021-04-25,0,0,0,0.0,0.0
1547343,1540659,Unassigned,Wyoming,90056,0,2021-04-26,0,0,0,0.0,0.0
1547344,1544001,Unassigned,Wyoming,90056,0,2021-04-27,0,0,0,0.0,0.0


In [20]:
cases_by_state_df.isna().sum()

Unnamed: 0       0
Countyname       0
ST_Name          0
FIPS             0
ST_ID            0
dt               0
Confirmed        0
Deaths           0
Population       0
IncidenceRate    0
NewCases         0
dtype: int64

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'